In [1]:
import pandas as pd
import pickle
from data import Dataset
import numpy as np

/Users/yhschan/miniforge3/envs/nlp/lib/python3.9/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
dataset = Dataset(dataset = 'toronto', model_type = 'gpt2', csv_file = "dataset/yelp_toronto_fix.csv", masking=False, max_len=400, reindex=False)

100%|██████████| 55966/55966 [00:28<00:00, 1934.74it/s]


In [3]:
unique, counts = np.unique(np.concatenate([dataset.user_labels_train, dataset.user_labels_eval, dataset.user_labels_test]), return_counts = True)

# Top 3 user index

In [4]:
top_3_indices = counts.argsort()[::-1][:3]
print(top_3_indices)

[125  69 160]


In [5]:
counts[top_3_indices[0]], counts[top_3_indices[1]], counts[top_3_indices[2]]

(536, 206, 201)

In [6]:
# Top 3: 125, 69, 160
for user_index in top_3_indices:
    print(np.sum(dataset.user_labels_train == user_index), np.sum(dataset.user_labels_eval == user_index), np.sum(dataset.user_labels_test == user_index))

427 49 60
170 21 15
162 20 19


In [7]:
selected_user1 = dataset.user_id_list[top_3_indices[0]]
selected_user2 = dataset.user_id_list[top_3_indices[1]]
selected_user3 = dataset.user_id_list[top_3_indices[2]]
print(selected_user1, selected_user2, selected_user3)

CxDOIDnH8gp9KXzpBHJYXw eZeBuiVZWT7u3SktO7mv9w JrXC_MDp38BWwLn2SFdNsA


In [8]:
print(len(dataset.df.query("user_id == @selected_user1")))
print(len(dataset.df.query("user_id == @selected_user2")))
print(len(dataset.df.query("user_id == @selected_user3")))

536
206
201


In [9]:
dataset.df.query("user_id == @selected_user1").to_csv("dataset/toronto/top3/selected_user1.csv")
dataset.df.query("user_id == @selected_user2").to_csv("dataset/toronto/top3/selected_user2.csv")
dataset.df.query("user_id == @selected_user3").to_csv("dataset/toronto/top3/selected_user3.csv")

# Last 3 user index

In [10]:
last_3_indices = counts.argsort()[:3]
print(last_3_indices)

[1061 1072 1071]


In [11]:
counts[last_3_indices[0]], counts[last_3_indices[1]], counts[last_3_indices[2]]

(7, 8, 8)

In [12]:
for user_index in last_3_indices:
    print(np.sum(dataset.user_labels_train == user_index), np.sum(dataset.user_labels_eval == user_index), np.sum(dataset.user_labels_test == user_index))

5 1 1
4 2 2
4 2 2


In [13]:
selected_user1 = dataset.user_id_list[last_3_indices[0]]
selected_user2 = dataset.user_id_list[last_3_indices[1]]
selected_user3 = dataset.user_id_list[last_3_indices[2]]
print(selected_user1, selected_user2, selected_user3)

UhlXS6NdA5k4SRunpcKHJw qoS4nIc5MlfF_j8DkYHBgw xHMq2fBArXBp881TVlRt5g


In [14]:
print(len(dataset.df.query("user_id == @selected_user1")))
print(len(dataset.df.query("user_id == @selected_user2")))
print(len(dataset.df.query("user_id == @selected_user3")))

7
8
8


In [15]:
dataset.df.query("user_id == @selected_user1").to_csv("dataset/toronto/last3/selected_user1.csv")
dataset.df.query("user_id == @selected_user2").to_csv("dataset/toronto/last3/selected_user2.csv")
dataset.df.query("user_id == @selected_user3").to_csv("dataset/toronto/last3/selected_user3.csv")

In [25]:
# Debug cell for resize function
dataset_user = Dataset(dataset = 'toronto', model_type = 'bert', csv_file = "dataset/toronto/top3/selected_user1.csv", masking=False, max_len=400, reindex=False)
# print((len(dataset_user.user_labels_train), len(dataset_user.user_labels_eval), len(dataset_user.user_labels_test)))
dataset_user.resize(427,49)
# len(dataset_user.user_labels_train), len(dataset_user.user_labels_eval), len(dataset_user.user_labels_test)
len(dataset_user.labels)

100%|██████████| 536/536 [00:00<00:00, 1957.82it/s]


536

# Get the exact same selected user dataset class

In [ ]:
import os
user_indices = list(top_3_indices) + list(last_3_indices)
i = 0
for dir in ['dataset/toronto/top3', 'dataset/toronto/last3']:
    for selected_user in ['selected_user1', 'selected_user2', 'selected_user3']:
        for model_type in ['bert', 'bart', 'gpt2']:
            csv_file = os.path.join(dir, selected_user) + '.csv'
            print(csv_file, model_type)
            dataset_user = Dataset(dataset = 'toronto', model_type = model_type, csv_file = csv_file, masking=False, max_len=400, reindex=False)

            old_train_len, old_eval_len, old_test_len = np.sum(dataset.user_labels_train == user_indices[i]), np.sum(dataset.user_labels_eval == user_indices[i]), np.sum(dataset.user_labels_test == user_indices[i])
            dataset_user.resize(old_train_len, old_eval_len)
            new_train_len, new_eval_len, new_test_len = len(dataset_user.user_labels_train), len(dataset_user.user_labels_eval), len(dataset_user.user_labels_test)

            assert (new_train_len, new_eval_len, new_test_len) == (old_train_len, old_eval_len, old_test_len)

            pickle_file = os.path.join(dir, selected_user) + f"_{model_type}.pkl" 
            with open(pickle_file, 'wb') as fout:
                pickle.dump(dataset_user, fout)
        i += 1